<a href="https://colab.research.google.com/github/DeepaliVerma/personal_files/blob/main/stylenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/kacky24/stylenet/blob/master/data_loader.py

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import re
import pickle
from collections import Counter
import nltk


class Vocab:
    '''vocabulary'''
    def __init__(self):
        self.w2i = {}
        self.i2w = {}
        self.ix = 0

    def add_word(self, word):
        if word not in self.w2i:
            self.w2i[word] = self.ix
            self.i2w[self.ix] = word
            self.ix += 1

    def __call__(self, word):
        if word not in self.w2i:
            return self.w2i['<unk>']
        return self.w2i[word]

    def __len__(self):
        return len(self.w2i)


def build_vocab(mode_list=['factual', 'humorous']):
    '''build vocabulary'''
    # define vocabulary
    vocab = Vocab()
    # add special tokens
    vocab.add_word('<pad>')
    vocab.add_word('<s>')
    vocab.add_word('</s>')
    vocab.add_word('<unk>')

    # add words
    for mode in mode_list:
        if mode == 'factual':
            captions = extract_captions(mode=mode)
            words = nltk.tokenize.word_tokenize(captions)
            counter = Counter(words)
            words = [word for word, cnt in counter.items() if cnt >= 2]
        else:
            captions = extract_captions(mode=mode)
            words = nltk.tokenize.word_tokenize(captions)

        for word in words:
            vocab.add_word(word)

    return vocab

# ls /content/drive/Shareddrives/BTP_group1/Flicker8k_Dataset/

def extract_captions(mode='factual'):
    '''extract captions from data files for building vocabulary'''
    text = ''
    if mode == 'factual':
        with open("/content/drive/Shareddrives/BTP_group1/Flickr8k_text/Flickr8k.token.txt", 'r') as f:
            res = f.readlines()

        r = re.compile(r'\d*.jpg#\d*')
        for line in res:
            line = r.sub('', line)
            line = line.replace('.', '')
            line = line.strip()
            text += line + ' '
# ls /content/drive/MyDrive/new_codes_github/FlickrStyle_v0.9/FlickrStyle_v0.9/romantic/romantic_train.txt
    else:
        if mode == 'humorous':
            with open("/content/drive/MyDrive/new_codes_github/FlickrStyle_v0.9/FlickrStyle_v0.9/humor/funny_train.txt", 'r') as f:
                res = f.readlines()
        else:
            with open("/content/drive/MyDrive/new_codes_github/FlickrStyle_v0.9/FlickrStyle_v0.9/romantic/romantic_train.txt", 'r') as f:
                res = f.readlines()

        for line in res:
            line = line.replace('.', '')
            line = line.strip()
            text += line + ' '

    return text.strip().lower()


if __name__ == '__main__':
    vocab = build_vocab(mode_list=['factual', 'humorous'])
    print(vocab.__len__())
    with open('/content/drive/MyDrive/new_codes_github/FlickrStyle_v0.9/vocab.pkl', 'wb') as f:
        pickle.dump(vocab, f)

16244


In [3]:
import os
import re
import pickle
import nltk
import skimage.io
import skimage.transform
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
# from build_vocab import Vocab


class Flickr7kDataset(Dataset):
    '''Flickr7k dataset'''
    def __init__(self, img_dir, caption_file, vocab, transform=None):
        '''
        Args:
            img_dir: Direcutory with all the images
            caption_file: Path to the factual caption file
            vocab: Vocab instance
            transform: Optional transform to be applied
        '''
        self.img_dir = img_dir
        # print("caption_file", caption_file)
        self.imgname_caption_list = self._get_imgname_and_caption(caption_file)
        self.vocab = vocab
        self.transform = transform

    def _get_imgname_and_caption(self, caption_file):
        '''extract image name and caption from factual caption file'''
        with open(caption_file, 'r') as f:
            # print("f",f)
            res = f.readlines()
            # print("res", res)

        imgname_caption_list = []
        r = re.compile(r'#\d*')
        for line in res:
            img_and_cap = r.split(line)
            img_and_cap = [x.strip() for x in img_and_cap]
            imgname_caption_list.append(img_and_cap)
            # print("img_and_cap", img_and_cap)
        return imgname_caption_list

    def __len__(self):
        return len(self.imgname_caption_list)

    def __getitem__(self, ix):
        '''return one data pair (image and captioin)'''
        img_name = self.imgname_caption_list[ix][0]
        img_name = os.path.join(self.img_dir, img_name)
        # print("self.imgname_caption_list:", self.imgname_caption_list[ix][1])
        caption = self.imgname_caption_list[ix][1]

        image = skimage.io.imread(img_name)
        if self.transform is not None:
            image = self.transform(image)

        # convert caption to word ids
        r = re.compile("\.")
        tokens = nltk.tokenize.word_tokenize(r.sub("", caption).lower())
        caption = []
        caption.append(self.vocab('<s>'))
        caption.extend([self.vocab(token) for token in tokens])
        caption.append(self.vocab('</s>'))
        caption = torch.Tensor(caption)
        return image, caption


class FlickrStyle7kDataset(Dataset):
    '''Styled caption dataset'''
    def __init__(self, caption_file, vocab):
        '''
        Args:
            caption_file: Path to styled caption file
            vocab: Vocab instance
        '''
        self.caption_list = self._get_caption(caption_file)
        self.vocab = vocab

    def _get_caption(self, caption_file):
        '''extract caption list from styled caption file'''
        with open(caption_file, 'r') as f:
            caption_list = f.readlines()

        caption_list = [x.strip() for x in caption_list]
        return caption_list

    def __len__(self):
        return len(self.caption_list)

    def __getitem__(self, ix):
        caption = self.caption_list[ix]
        # convert caption to word ids
        r = re.compile("\.")
        tokens = nltk.tokenize.word_tokenize(r.sub("", caption).lower())
        caption = []
        caption.append(self.vocab('<s>'))
        caption.extend([self.vocab(token) for token in tokens])
        caption.append(self.vocab('</s>'))
        caption = torch.Tensor(caption)
        return caption


def get_data_loader(img_dir, caption_file, vocab, batch_size,
                    transform=None, shuffle=False, num_workers=0):
    '''Return data_loader'''
    if transform is None:
        transform = transforms.Compose([
            Rescale((224, 224)),
            transforms.ToTensor()
            ])
    # print("caption_file", caption_file)
    flickr7k = Flickr7kDataset(img_dir, caption_file, vocab, transform)
    # print("flickr7k", flickr7k)
    data_loader = DataLoader(dataset=flickr7k,
                             batch_size=batch_size,
                             shuffle=shuffle,
                             num_workers=num_workers,
                             collate_fn=collate_fn)
    return data_loader


def get_styled_data_loader(caption_file, vocab, batch_size,
                           shuffle=False, num_workers=0):
    '''Return data_loader for styled caption'''
    flickr_styled_7k = FlickrStyle7kDataset(caption_file, vocab)

    data_loader = DataLoader(dataset=flickr_styled_7k,
                             batch_size=batch_size,
                             shuffle=shuffle,
                             num_workers=num_workers,
                             collate_fn=collate_fn_styled)
    return data_loader


class Rescale:
    '''Rescale the image to a given size
    Args:
        output_size(int or tuple)
    '''
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, image):
        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)
        image = skimage.transform.resize(image, (new_h, new_w))

        return image


def collate_fn(data):
    '''create minibatch tensors from data(list of tuple(image, caption))'''
    data.sort(key=lambda x: len(x[1]), reverse=True)
    images, captions = zip(*data)

    # images : tuple of 3D tensor -> 4D tensor
    images = torch.stack(images, 0)

    # captions : tuple of 1D Tensor -> 2D tensor
    lengths = torch.LongTensor([len(cap) for cap in captions])
    captions = [pad_sequence(cap, max(lengths)) for cap in captions]
    captions = torch.stack(captions, 0)

    return images, captions, lengths


def collate_fn_styled(captions):
    captions.sort(key=lambda x: len(x), reverse=True)

    # tuple of 1D Tensor -> 2D Tensor
    lengths = torch.LongTensor([len(cap) for cap in captions])
    captions = [pad_sequence(cap, max(lengths)) for cap in captions]
    captions = torch.stack(captions, 0)

    return captions, lengths


def pad_sequence(seq, max_len):
    seq = torch.cat((seq, torch.zeros(max_len - len(seq))))
    return seq


if __name__ == "__main__":
    with open("/content/drive/MyDrive/new_codes_github/FlickrStyle_v0.9/vocab.pkl", 'rb') as f:
        vocab = pickle.load(f)

    img_path = "/content/drive/Shareddrives/BTP_group1/Flicker8k_Dataset/"
    cap_path = "/content/drive/Shareddrives/BTP_group1/Flickr8k_text/Flickr8k.token.txt"
    cap_path_styled = "/content/drive/MyDrive/new_codes_github/FlickrStyle_v0.9/FlickrStyle_v0.9/humor/funny_train.txt"
    data_loader = get_data_loader(img_path, cap_path, vocab, 3)
    styled_data_loader = get_styled_data_loader(cap_path_styled, vocab, 3)

    for i, (img, captions, lengths) in enumerate(data_loader):
        print("i------->",i)
        # print(images.shape)
        print("caption----------->",captions[:, 1:])
        print("length--------------->",lengths - 1)
        print()
        if i == 3:
            break

    for i, (captions, lengths) in enumerate(styled_data_loader):
        print("i------->",i)
        # print(images.shape)
        print("caption----------->",captions[:, 1:])
        print("length--------------->",lengths - 1)
        print()
        if i == 3:
            break

i-------> 0
caption-----------> tensor([[ 5.,  6.,  7.,  5.,  8.,  9., 10., 11., 12.,  5., 13., 14., 15.,  7.,
         16.,  3., 17.,  2.],
        [ 5., 23., 18., 11., 20.,  5., 21., 24.,  2.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.],
        [ 5., 18., 19., 20.,  5., 21., 22.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.]])
length---------------> tensor([18,  9,  8])

i-------> 1
caption-----------> tensor([[ 5., 23., 18.,  7.,  5.,  8.,  9., 19., 20.,  5., 21., 28.,  2.],
        [ 5., 23., 18., 11., 25., 15., 26., 27., 24.,  2.,  0.,  0.,  0.],
        [ 5., 30., 31., 32.,  5., 33., 31., 34., 35.,  2.,  0.,  0.,  0.]])
length---------------> tensor([13, 10, 10])

i-------> 2
caption-----------> tensor([[ 5., 30., 31., 32.,  5., 43., 31., 38., 44., 45., 34., 46., 47., 39.,
         40.,  7., 25., 48.,  2.],
        [ 5., 30., 31., 32.,  5., 36., 31., 37., 38., 39., 40., 41., 25., 42.,
          2.,  0.,  0.,  0.,  0.],
        [49., 50., 14., 51., 52.

In [4]:
SYMBOL_MAP = {
        '<pad>': 0,
        '<s>': 1,
        '</s>': 2,
        '<unk>': 3
        }


def get_symbol_id(symbol):
    return SYMBOL_MAP[symbol]


In [5]:
import sys
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torch.autograd import Variable
# from constant import get_symbol_id


class EncoderCNN(nn.Module):
    def __init__(self, emb_dim):
        '''
        Load the pretrained ResNet152 and replace fc
        '''
        super(EncoderCNN, self).__init__()
        resnet = models.resnet152(pretrained=True)
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        self.A = nn.Linear(resnet.fc.in_features, emb_dim)

    def forward(self, images):
        '''Extract the image feature vectors'''
        features = self.resnet(images)
        features = Variable(features.data)
        # if torch.cuda.is_available():
        #     features = features.cuda()
        features = features.view(features.size(0), -1)
        features = self.A(features)
        return features


class FactoredLSTM(nn.Module):
    def __init__(self, emb_dim, hidden_dim, factored_dim,  vocab_size):
        super(FactoredLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size

        # embedding
        self.B = nn.Embedding(vocab_size, emb_dim)

        # factored lstm weights
        self.U_i = nn.Linear(factored_dim, hidden_dim)
        self.S_fi = nn.Linear(factored_dim, factored_dim)
        self.V_i = nn.Linear(emb_dim, factored_dim)
        self.W_i = nn.Linear(hidden_dim, hidden_dim)

        self.U_f = nn.Linear(factored_dim, hidden_dim)
        self.S_ff = nn.Linear(factored_dim, factored_dim)
        self.V_f = nn.Linear(emb_dim, factored_dim)
        self.W_f = nn.Linear(hidden_dim, hidden_dim)

        self.U_o = nn.Linear(factored_dim, hidden_dim)
        self.S_fo = nn.Linear(factored_dim, factored_dim)
        self.V_o = nn.Linear(emb_dim, factored_dim)
        self.W_o = nn.Linear(hidden_dim, hidden_dim)

        self.U_c = nn.Linear(factored_dim, hidden_dim)
        self.S_fc = nn.Linear(factored_dim, factored_dim)
        self.V_c = nn.Linear(emb_dim, factored_dim)
        self.W_c = nn.Linear(hidden_dim, hidden_dim)

        self.S_hi = nn.Linear(factored_dim, factored_dim)
        self.S_hf = nn.Linear(factored_dim, factored_dim)
        self.S_ho = nn.Linear(factored_dim, factored_dim)
        self.S_hc = nn.Linear(factored_dim, factored_dim)

        # self.S_ri = nn.Linear(factored_dim, factored_dim)
        # self.S_rf = nn.Linear(factored_dim, factored_dim)
        # self.S_ro = nn.Linear(factored_dim, factored_dim)
        # self.S_rc = nn.Linear(factored_dim, factored_dim)

        # weight for output
        self.C = nn.Linear(hidden_dim, vocab_size)

    def forward_step(self, embedded, h_0, c_0, mode):
        i = self.V_i(embedded)
        f = self.V_f(embedded)
        o = self.V_o(embedded)
        c = self.V_c(embedded)

        if mode == "factual":
            i = self.S_fi(i)
            f = self.S_ff(f)
            o = self.S_fo(o)
            c = self.S_fc(c)
        elif mode == "humorous":
            i = self.S_hi(i)
            f = self.S_hf(f)
            o = self.S_ho(o)
            c = self.S_hc(c)
        # elif mode == "romantic":
        #     i = self.S_ri(i)
        #     f = self.S_rf(f)
        #     o = self.S_ro(o)
        #     c = self.S_rc(c)
        else:
            sys.stderr.write("mode name wrong!")

        i_t = F.sigmoid(self.U_i(i) + self.W_i(h_0))
        f_t = F.sigmoid(self.U_f(f) + self.W_f(h_0))
        o_t = F.sigmoid(self.U_o(o) + self.W_o(h_0))
        c_tilda = F.tanh(self.U_c(c) + self.W_c(h_0))

        c_t = f_t * c_0 + i_t * c_tilda
        h_t = o_t * c_t

        outputs = self.C(h_t)

        return outputs, h_t, c_t

    def forward(self, captions, features=None, mode="factual"):
        '''
        Args:
            features: fixed vectors from images, [batch, emb_dim]
            captions: [batch, max_len]
            mode: type of caption to generate
        '''
        batch_size = captions.size(0)
        embedded = self.B(captions)  # [batch, max_len, emb_dim]
        # concat features and captions
        if mode == "factual":
            if features is None:
                sys.stderr.write("features is None!")
            embedded = torch.cat((features.unsqueeze(1), embedded), 1)

        # initialize hidden state
        h_t = Variable(torch.Tensor(batch_size, self.hidden_dim))
        c_t = Variable(torch.Tensor(batch_size, self.hidden_dim))
        nn.init.uniform(h_t)
        nn.init.uniform(c_t)

        if torch.cuda.is_available():
            h_t = h_t.cuda()
            c_t = c_t.cuda()

        all_outputs = []
        # iterate
        for ix in range(embedded.size(1) - 1):
            emb = embedded[:, ix, :]
            outputs, h_t, c_t = self.forward_step(emb, h_t, c_t, mode=mode)
            all_outputs.append(outputs)

        all_outputs = torch.stack(all_outputs, 1)

        return all_outputs

    def sample(self, feature, beam_size=5, max_len=30, mode="factual"):
        '''
        generate captions from feature vectors with beam search
        Args:
            features: fixed vector for an image, [1, emb_dim]
            beam_size: stock size for beam search
            max_len: max sampling length
            mode: type of caption to generate
        '''
        # initialize hidden state
        h_t = Variable(torch.Tensor(1, self.hidden_dim))
        c_t = Variable(torch.Tensor(1, self.hidden_dim))
        nn.init.uniform(h_t)
        nn.init.uniform(c_t)

        # if torch.cuda.is_available():
        #     h_t = h_t.cuda()
        #     c_t = c_t.cuda()

        # forward 1 step
        _, h_t, c_t = self.forward_step(feature, h_t, c_t, mode=mode)

        # candidates: [score, decoded_sequence, h_t, c_t]
        symbol_id = torch.LongTensor([1]).unsqueeze(0)
        symbol_id = Variable(symbol_id, volatile=True)
        # if torch.cuda.is_available():
        #     symbol_id = symbol_id.cuda()
        candidates = [[0, symbol_id, h_t, c_t, [get_symbol_id('<s>')]]]

        # beam search
        t = 0
        while t < max_len - 1:
            t += 1
            tmp_candidates = []
            end_flag = True
            for score, last_id, h_t, c_t, id_seq in candidates:
                if id_seq[-1] == get_symbol_id('</s>'):
                    tmp_candidates.append([score, last_id, h_t, c_t, id_seq])
                else:
                    end_flag = False
                    emb = self.B(last_id)
                    output, h_t, c_t = self.forward_step(emb, h_t, c_t, mode=mode)
                    output = output.squeeze(0).squeeze(0)
                    # log softmax
                    output = F.log_softmax(output)
                    output, indices = torch.sort(output, descending=True)
                    output = output[:beam_size]
                    indices = indices[:beam_size]
                    score_list = score + output
                    for score, wid in zip(score_list, indices):
                        tmp_candidates.append(
                            [score, wid, h_t, c_t, id_seq + [int(wid.data.numpy())]]
                        )
            if end_flag:
                break
            # sort by normarized log probs and pick beam_size highest candidate
            candidates = sorted(tmp_candidates,
                                key=lambda x: -x[0].data.numpy()/len(x[-1]))[:beam_size]

        return candidates[0][-1]

In [6]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable


def sequence_mask(sequence_length, max_len=None):
    if max_len is None:
        max_len = sequence_length.data.max()
    batch_size = sequence_length.size(0)
    seq_range = torch.range(0, max_len - 1).long()
    seq_range_expand = seq_range.unsqueeze(0).expand(batch_size, max_len)
    seq_range_expand = Variable(seq_range_expand)
    if sequence_length.is_cuda:
        seq_range_expand = seq_range_expand.cuda()
    seq_length_expand = (sequence_length.unsqueeze(1)
                         .expand_as(seq_range_expand))
    return seq_range_expand < seq_length_expand


def masked_cross_entropy(logits, target, length):
    length = Variable(length)
    if torch.cuda.is_available():
        length = length.cuda()

    """
    Args:
        logits: A Variable containing a FloatTensor of size
            (batch, max_len, num_classes) which contains the
            unnormalized probability for each class.
        target: A Variable containing a LongTensor of size
            (batch, max_len) which contains the index of the true
            class for each corresponding step.
        length: A Variable containing a LongTensor of size (batch,)
            which contains the length of each data in a batch.
    Returns:
        loss: An average loss value masked by the length.
    """

    # logits_flat: (batch * max_len, num_classes)
    logits_flat = logits.view(-1, logits.size(-1))
    # log_probs_flat: (batch * max_len, num_classes)
    log_probs_flat = F.log_softmax(logits_flat)
    # target_flat: (batch * max_len, 1)
    target_flat = target.view(-1, 1)
    # losses_flat: (batch * max_len, 1)
    losses_flat = -torch.gather(log_probs_flat, dim=1, index=target_flat)
    # losses: (batch, max_len)
    losses = losses_flat.view(*target.size())
    # mask: (batch, max_len)
    mask = sequence_mask(sequence_length=length, max_len=target.size(1))
    losses = losses * mask.float()
    loss = losses.sum() / length.float().sum()
    return loss


if __name__ == '__main__':
    length = torch.LongTensor([23, 21, 17])
    length = Variable(length)

    print(sequence_mask(length))

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True, False, False, False,
         False, False, False]])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  # Remove the CWD from sys.path while we load stuff.


In [7]:
import os
import pickle
import argparse
import torch
from torch.autograd import Variable
# from build_vocab import Vocab
# from data_loader import get_data_loader
# from data_loader import get_styled_data_loader
# from models import EncoderCNN
# from models import FactoredLSTM
# from loss import masked_cross_entropy

os.environ['CUDA_VISIBLE_DEVICES'] = '1'


def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)


def eval_outputs(outputs, vocab):
    # outputs: [batch, max_len - 1, vocab_size]
    indices = torch.topk(outputs, 1)[1]
    indices = indices.squeeze(2)
    indices = indices.data
    for i in range(len(indices)):
        caption = [vocab.i2w[x] for x in indices[i]]
        print(caption)


def main( ):

    model_path = '/content/drive/MyDrive/new_codes_github/FlickrStyle_v0.9/'
    if not os.path.exists(model_path):
        os.makedirs(model_path)

    # load vocablary
    vocab_path='/content/drive/MyDrive/new_codes_github/FlickrStyle_v0.9/vocab.pkl'
    with open(vocab_path, 'rb') as f:
        vocab = pickle.load(f)

    img_path = "/content/drive/Shareddrives/BTP_group1/Flicker8k_Dataset/"
    factual_cap_path = "/content/drive/Shareddrives/BTP_group1/Flickr8k_text/Flickr8k.token.txt"
    humorous_cap_path = "/content/drive/MyDrive/new_codes_github/FlickrStyle_v0.9/FlickrStyle_v0.9/humor/funny_train.txt"
    # img_path = args.img_path
    # factual_cap_path = args.factual_caption_path
    # humorous_cap_path = args.humorous_caption_path
    caption_batch_size=64
    language_batch_size=96
    emb_dim=300
    hidden_dim=512
    factored_dim=512
    lr_caption=0.0002
    lr_language=0.0005
    epoch_num=30
    log_step_caption=50
    log_step_language=10



    # import data_loader
    data_loader = get_data_loader(img_path, factual_cap_path, vocab,caption_batch_size, shuffle=True)
    # print(" factual_cap_path", factual_cap_path)
    styled_data_loader = get_styled_data_loader(humorous_cap_path, vocab,
                                                language_batch_size,
                                                shuffle=True)
    # img_path= next(iter(data_loader))
    # print(f"Feature batch shape: {img_path.size()}")
    # print(f"Labels batch shape: {factual_cap_path.size()}")
    
    emb_dim = emb_dim
    hidden_dim = hidden_dim
    factored_dim = factored_dim
    vocab_size = len(vocab)
    encoder = EncoderCNN(emb_dim)
    decoder = FactoredLSTM(emb_dim, hidden_dim, factored_dim, vocab_size)

    if torch.cuda.is_available():
        encoder = encoder.cuda()
        decoder = decoder.cuda()

    # loss and optimizer
    criterion = masked_cross_entropy
    cap_params = list(decoder.parameters()) + list(encoder.A.parameters())
    lang_params = list(decoder.parameters())
    optimizer_cap = torch.optim.Adam(cap_params, lr=lr_caption)
    optimizer_lang = torch.optim.Adam(lang_params, lr=lr_language)

    # train
    total_cap_step = len(data_loader)
    print("total_cap_step", total_cap_step)
    total_lang_step = len(styled_data_loader)
    print("total_lang_step", total_lang_step)
    epoch_num = epoch_num
    for epoch in range(epoch_num):
        # caption
        # for i, (images, captions, lengths) in enumerate(data_loader):
        for i, (images, captions, lengths) in enumerate(data_loader):
            images = to_var(images, volatile=True)
            captions = to_var(captions.long())

            # forward, backward and optimize
            decoder.zero_grad()
            encoder.zero_grad()
            features = encoder(images.float())
            outputs = decoder(captions, features, mode="factual")
            loss = criterion(outputs[:, 1:, :].contiguous(),
                             captions[:, 1:].contiguous(), lengths - 1)
            loss.backward()
            optimizer_cap.step()

            # print log
            if i % log_step_caption == 0:
                print("Epoch [%d/%d], CAP, Step [%d/%d], Loss: %.4f"
                      % (epoch+1, epoch_num, i, total_cap_step,
                          loss.data.mean()))

        eval_outputs(outputs, vocab)

        # language
        for i, (captions, lengths) in enumerate(styled_data_loader):
            captions = to_var(captions.long())

            # forward, backward and optimize
            decoder.zero_grad()
            outputs = decoder(captions, mode='humorous')
            loss = criterion(outputs, captions[:, 1:].contiguous(), lengths-1)
            loss.backward()
            optimizer_lang.step()

            # print log
            if i % log_step_language == 0:
                print("Epoch [%d/%d], LANG, Step [%d/%d], Loss: %.4f"
                      % (epoch+1, epoch_num, i, total_lang_step,
                          loss.data.mean()))

        # save models
        torch.save(decoder.state_dict(),
                   os.path.join(model_path, 'decoder-%d.pkl' % (epoch + 1,)))

        torch.save(encoder.state_dict(),
                   os.path.join(model_path, 'encoder-%d.pkl' % (epoch + 1,)))


if __name__ == '__main__':
    # parser = argparse.ArgumentParser(
    #         description='StyleNet: Generating Attractive Visual Captions \
    #                     with Styles')
    # parser.add_argument('--model_path', type=str, default='pretrained_models',
    #                     help='path for saving trained models')
    # parser.add_argument('--vocab_path', type=str, default='./data/vocab.pkl',
    #                     help='path for vocabrary')
    # parser.add_argument('--img_path', type=str,
    #                     default='./data/flickr7k_images',
    #                     help='path for train images directory')
    # parser.add_argument('--factual_caption_path', type=str,
    #                     default='./data/factual_train.txt',
    #                     help='path for factual caption file')
    # parser.add_argument('--humorous_caption_path', type=str,
    #                     default='./data/humor/funny_train.txt',
    #                     help='path for humorous caption file')
    # parser.add_argument('--romantic_caption_path', type=str,
    #                     default='./data/romantic/romanntic_train.txt',
    #                     help='path for romantic caption file')
    # parser.add_argument('--caption_batch_size', type=int, default=64,
    #                     help='mini batch size for caption model training')
    # parser.add_argument('--language_batch_size', type=int, default=96,
    #                     help='mini batch size for language model training')
    # parser.add_argument('--emb_dim', type=int, default=300,
    #                     help='embedding size of word, image')
    # parser.add_argument('--hidden_dim', type=int, default=512,
    #                     help='hidden state size of factored LSTM')
    # parser.add_argument('--factored_dim', type=int, default=512,
    #                     help='size of factored matrix')
    # parser.add_argument('--lr_caption', type=int, default=0.0002,
    #                     help='learning rate for caption model training')
    # parser.add_argument('--lr_language', type=int, default=0.0005,
    #                     help='learning rate for language model training')
    # parser.add_argument('--epoch_num', type=int, default=30)
    # parser.add_argument('--log_step_caption', type=int, default=50,
    #                     help='steps for print log while train caption model')
    # parser.add_argument('--log_step_language', type=int, default=10,
    #                     help='steps for print log while train language model')
    # args = parser.parse_args()

    main()

total_cap_step 633
total_lang_step 73


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:129: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:130: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warn

Epoch [1/30], CAP, Step [0/633], Loss: 9.7000
Epoch [1/30], CAP, Step [50/633], Loss: 5.0705
Epoch [1/30], CAP, Step [100/633], Loss: 4.6606
Epoch [1/30], CAP, Step [150/633], Loss: 4.3959
Epoch [1/30], CAP, Step [200/633], Loss: 4.1278
Epoch [1/30], CAP, Step [250/633], Loss: 4.2035


FileNotFoundError: ignored